In [1]:
from __future__ import print_function

from IPython.display import HTML, display

import os

import numpy as np
import pandas as pd
from root_pandas import read_root, to_root

import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.base import BaseEstimator
from sklearn.grid_search import RandomizedSearchCV

from xgboost import XGBClassifier

from itertools import islice

from tqdm import tqdm

import ROOT

from uncertainties import ufloat

from collections import OrderedDict
from scripts.metrics import tagging_power_score

In [2]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 14

# Read and prepare Data

In [3]:
data_dir = '/home/kheinicke/tank/flavourtagging/'
filenames = [
    data_dir + 'Bu2JpsiK_mu-k-e-TrainingTuple_2011_MD_sweighted_kheinick.root',
    data_dir + 'Bu2JpsiK_mu-k-e-TrainingTuple_2011_MU_sweighted_kheinick.root',
    data_dir + 'Bu2JpsiK_mu-k-e-TrainingTuple_2012_MD_sweighted_kheinick.root',
    data_dir + 'Bu2JpsiK_mu-k-e-TrainingTuple_2012_MU_sweighted_kheinick.root',
]
chunksize = 5000

In [4]:
# just define some keyword arguments for read_root in a separate dict
data_kwargs = dict(
    key='DecayTree',  # the tree name
    columns=['B_OS_Muon*',  # all branches that should be read
             'B_ID',
             'B_PT',
             'SigYield_sw',
             'runNumber',
             'eventNumber',
            ],
    chunksize=chunksize,  # this will create a generator, yielding subsets with 'chunksize' of the data
    where='(B_LOKI_MASS_JpsiConstr_NoPVConstr>0)',  # a ROOT where selection, does not work with array-variables
    flatten=True  # will flatten the data in the dimension of the first given column
)

In [5]:
# still have to use plain ROOT to get the number of entries...
n_entries = 0
for fn in filenames:
    f = ROOT.TFile(fn)
    t = f.Get('DecayTree')
    n_entries += t.GetEntries()

In [6]:
# This will read chunks of the data inside a list comprehension and then concat those to a big dataframe
# note that tqdm is just some boilerplate to generate a progressbar
# df = pd.concat([df for df in tqdm(read_root(filenames, **data_kwargs), total=n_entries/chunksize)])

# only use the first 10 chunks to speed up read process and reduce RAM pressure for development
df = pd.concat([df for df in tqdm(islice(read_root(filenames, **data_kwargs), 30), total=30)])

100%|██████████| 30/30 [00:30<00:00,  1.01s/it]


In [7]:
df['target'] = np.sign(df.B_ID) == np.sign(df.B_OS_Muon_ID)
df.rename(columns=dict(zip(df.columns, [c.replace('B_OS_Muon', 'tp') for c in df.columns])), inplace=True)
df['tp_ABS_RecVertexIP'] = np.abs(df.tp_RecVertexIP)
df['event_id'] = df.runNumber.apply(str) + '_' + df.eventNumber.apply(str)

In [8]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3874783 entries, 0 to 127832
Data columns (total 39 columns):
tp_DEC                 int32
tp_PROB                float64
tp_PARTICLES_NUM       int32
tp_ABSID               float32
tp_partP               float32
tp_partPt              float32
tp_partlcs             float32
tp_BPVIPCHI2           float32
tp_TRTYPE              float32
tp_PIDmu               float32
tp_IPPU                float32
tp_ghostProb           float32
tp_PIDNNm              float32
tp_PROBNNpi            float32
tp_PROBNNe             float32
tp_PROBNNk             float32
tp_PROBNNp             float32
tp_PP_HASMUONPID       float32
tp_PP_MuonPIDStatus    float32
tp_IsSignalDaughter    float32
tp_Signal_P            float32
tp_minPhiDistance      float32
tp_MuonPIDIsMuon       float32
tp_RecVertexIP         float32
tp_mult                float32
tp_ptB                 float32
tp_IPs                 float32
tp_KEY                 float32
tp_Q                   f

# Build the meta classifier

In [9]:
# this is the list of BDT variables formerly used
classic_MVA_features = ['tp_' + c for c in [
    'partP',
    'partPt',
    'IPPU',
    'ghostProb',
    'PIDNNm',
    'ABS_RecVertexIP',
    'mult',
    'ptB',
    'IPs',
]]

In [10]:
def get_event_number(df, weight_column='SigYield_sw'):
    """ Use weighted sums
    """
    return np.sum(df.groupby('event_id')[weight_column].first())  # max, min, mean, first should give the same values here

In [11]:
class CutBasedXGBClassifier(XGBClassifier):
    def __init__(self, max_depth=3, learning_rate=0.1, n_estimators=100,
                 silent=True, objective="reg:linear",
                 nthread=-1, gamma=0, min_child_weight=1, max_delta_step=0,
                 subsample=1, colsample_bytree=1, colsample_bylevel=1,
                 reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                 base_score=0.5, seed=0, missing=None,
                 P_column='tp_partP', P_cut=0,
                 PT_column='tp_partPt', PT_cut=1.1,
                 phiDistance_column='tp_minPhiDistance', phiDistance_cut=0.005,
                 MuonPIDIsMuon_column='tp_MuonPIDIsMuon', MuonPIDIsMuon_cut=1,
                 mvaFeatures=None, only_max_pt=True, event_identifier_column='event_id',
                 IsSignalDaughter_column='tp_IsSignalDaughter', IsSignalDaughter_cut=0,
                 RecVertexIPs_column='tp_ABS_RecVertexIP', RecVertexIPs_cut=0,
                 TRCHI2DOF_column='tp_partlcs', TRCHI2DOF_cut=3,
                 TRGHP_column='tp_ghostProb', TRGHP_cut=0.4,
                 PROBNNmu_column='tp_PIDNNm', PROBNNmu_cut=0.35,
                 PROBNNpi_column='tp_PROBNNpi', PROBNNpi_cut=0.8,
                 PROBNNe_column='tp_PROBNNe', PROBNNe_cut=0.8,
                 PROBNNk_column='tp_PROBNNk', PROBNNk_cut=0.8,
                 PROBNNp_column='tp_PROBNNp', PROBNNp_cut=0.8,
                 IPPUs_column='tp_IPPU', IPPUs_cut=3,
                ):
        self.cut_parameters = ['P', 'PT', 'phiDistance', 'MuonPIDIsMuon',
                               'IsSignalDaughter', 'TRCHI2DOF', 'TRGHP',
                               'PROBNNmu', 'PROBNNpi', 'PROBNNe', 'PROBNNk',
                               'PROBNNp', 'IPPUs', 'RecVertexIPs',
                              ]
        for cp in self.cut_parameters:
            setattr(self, '{}_cut'.format(cp), locals()['{}_cut'.format(cp)])
            setattr(self, '{}_column'.format(cp), locals()['{}_column'.format(cp)])
        self.mvaFeatures = mvaFeatures
        self.only_max_pt = only_max_pt
        self.event_identifier_column = event_identifier_column
        self.fit_status_ = True
        super(CutBasedXGBClassifier, self).__init__(max_depth=max_depth, learning_rate=learning_rate,
                                                    n_estimators=n_estimators, silent=silent, objective=objective,
                                                    nthread=nthread, gamma=gamma, min_child_weight=min_child_weight,
                                                    max_delta_step=max_delta_step,
                                                    subsample=subsample, colsample_bytree=colsample_bytree,
                                                    colsample_bylevel=colsample_bylevel,
                                                    reg_alpha=reg_alpha, reg_lambda=reg_lambda,
                                                    scale_pos_weight=scale_pos_weight,
                                                    base_score=base_score, seed=seed, missing=None)

    def select(self, X, y=None):
        print('Applying selection')
        len_before = get_event_number(X)
        selection = ((X[self.P_column] > self.P_cut)
                     & (X[self.PT_column] > self.PT_cut)
                     & (X[self.phiDistance_column] > self.phiDistance_cut)
                     & (X[self.MuonPIDIsMuon_column] == self.MuonPIDIsMuon_cut)
                     & (X[self.IsSignalDaughter_column] == self.IsSignalDaughter_cut)
                     & (X[self.TRCHI2DOF_column] < self.TRCHI2DOF_cut)
                     & (X[self.TRGHP_column] < self.TRGHP_cut)
                     & (X[self.PROBNNmu_column] > self.PROBNNmu_cut)
                     & (X[self.PROBNNpi_column] < self.PROBNNpi_cut)
                     & (X[self.PROBNNe_column] < self.PROBNNe_cut)
                     & (X[self.PROBNNk_column] < self.PROBNNk_cut)
                     & (X[self.PROBNNp_column] < self.PROBNNp_cut)
                     & (X[self.IPPUs_column] > self.IPPUs_cut)
                     & (X[self.RecVertexIPs_column] > self.RecVertexIPs_cut)
                    )
        X = X[selection]
        if y is not None:
            y = y[selection]
        
        if self.only_max_pt:
            X.reset_index(drop=True, inplace=True)
            max_pt_indices = X.groupby(self.event_identifier_column)[self.PT_column].idxmax()
            X = X.iloc[max_pt_indices]
            if y is not None:
                y.reset_index(drop=True, inplace=True)
                y = y.iloc[max_pt_indices]

        len_after = get_event_number(X)
        self.efficiency_ = len_after / len_before

        if self.mvaFeatures:
            X = X[self.mvaFeatures]
        
        if y is not None:
            return X, y
        else:
            return X

    def get_params(self, deep=False):
        params = super(CutBasedXGBClassifier, self).get_params(deep=deep)
        for cp in self.cut_parameters:
            params['{}_cut'.format(cp)] = getattr(self, '{}_cut'.format(cp))
            params['{}_column'.format(cp)] = getattr(self, '{}_column'.format(cp))
        params['mvaFeatures'] = self.mvaFeatures
        params['only_max_pt'] = self.only_max_pt
        params['event_identifier_column'] = self.event_identifier_column
        return params

    def set_params(self, **kwargs):
        for cp in self.cut_parameters:
            cutname = '{}_cut'.format(cp)
            if kwargs.has_key(cutname) and kwargs[cutname] is not None:
                setattr(self, cutname, kwargs.pop(cutname))
        for other_param in ['mvaFeatures', 'only_max_pt', 'event_identifier_column']:
            if kwargs.has_key(other_param) and kwargs[other_param] is not None:
                setattr(self, other_param, kwargs.pop(other_param))
        super(CutBasedXGBClassifier, self).set_params(**kwargs)
        return self

    def fit(self, X, y, eval_set=None, **kwargs):
        if eval_set is not None:
            eval_set = [self.select(X_, y_) for X_, y_ in eval_set]
        X_, y_ = self.select(X, y)
        return super(CutBasedXGBClassifier, self).fit(X_, y_, eval_set=eval_set, **kwargs)

    def predict_proba(self, data, **kwargs):
        print('Predicting probas')
        return super(CutBasedXGBClassifier, self).predict_proba(self.select(data), **kwargs)

    def score(self, X, y, sample_weight=None):
        print('Calculating tagging power')
        probas = self.predict_proba(X)[:,1]
        sc = tagging_power_score(probas, efficiency=self.efficiency_, sample_weight=sample_weight)
        return sc

In [12]:
class SelectionKFold(KFold):
    def __init__(self, y, n_folds=3, shuffle=False, random_state=None):
        self.y = y
        self.unique_events = self.y.event_id.unique()
        self.raw_indices = np.arange(len(y))
        super(SelectionKFold, self).__init__(len(np.unique(y.event_id)), n_folds=n_folds,
                                             shuffle=shuffle, random_state=random_state)
    
    def __iter__(self):
        for train_indices, test_indices in super(SelectionKFold, self).__iter__():
            print('Yielding split')
            yield (self.raw_indices[self.y.event_id.isin(self.unique_events[train_indices]).values],
                   self.raw_indices[self.y.event_id.isin(self.unique_events[test_indices]).values])

In [13]:
df.event_id.unique()

array(['103146_64788876', '103146_7913757', '103146_66250073', ...,
       '97354_2562534720', '97354_2570553414', '97354_2577687734'], dtype=object)

In [18]:
skf = SelectionKFold(df, n_folds=2)

In [14]:
%%time
for a in skf:
    print(a)

Yielding split
(array([16103434, 16103435, 16103436, ..., 32726568, 32726569, 32726570]), array([       0,        1,        2, ..., 16103431, 16103432, 16103433]))
Yielding split
(array([       0,        1,        2, ..., 16103431, 16103432, 16103433]), array([16103434, 16103435, 16103436, ..., 32726568, 32726569, 32726570]))
CPU times: user 10.4 s, sys: 0 ns, total: 10.4 s
Wall time: 10.4 s


In [14]:
cbxgb = CutBasedXGBClassifier(mvaFeatures=classic_MVA_features, max_depth=5,
                              n_estimators=300, learning_rate=0.01, seed=10,
#                               **best_params
                             )

In [15]:
cbxgb.fit(df, df.target)

Applying selection


CutBasedXGBClassifier(IPPUs_column='tp_IPPU', IPPUs_cut=3,
           IsSignalDaughter_column='tp_IsSignalDaughter',
           IsSignalDaughter_cut=0, MuonPIDIsMuon_column='tp_MuonPIDIsMuon',
           MuonPIDIsMuon_cut=1, PROBNNe_column='tp_PROBNNe',
           PROBNNe_cut=0.8, PROBNNk_column='tp_PROBNNk', PROBNNk_cut=0.8,
           PROBNNmu_column='tp_PIDNNm', PROBNNmu_cut=0.35,
           PROBNNp_column='tp_PROBNNp', PROBNNp_cut=0.8,
           PROBNNpi_column='tp_PROBNNpi', PROBNNpi_cut=0.8,
           PT_column='tp_partPt', PT_cut=1.1, P_column='tp_partP', P_cut=0,
           RecVertexIPs_column='tp_ABS_RecVertexIP', RecVertexIPs_cut=0,
           TRCHI2DOF_column='tp_partlcs', TRCHI2DOF_cut=3,
           TRGHP_column='tp_ghostProb', TRGHP_cut=0.4, base_score=0.5,
           colsample_bylevel=1, colsample_bytree=1,
           event_identifier_column='event_id', gamma=0, learning_rate=0.01,
           max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
           mv

In [16]:
print('tagging power = {:.2f}%'.format(cbxgb.score(df, df.target) * 100))
# print('tagging power test = {:.2f}%'.format(cbxgb.score(test_data, test_labels) * 100))
# print('tagging power train = {:.2f}%'.format(cbxgb.score(train_data, train_labels) * 100))

Calculating tagging power
Predicting probas
Applying selection
tagging power = 0.74%


In [64]:
id(skf.y)

140242835526352

In [30]:
a = df.iloc[:1000]

In [62]:
id(a)

140241657959184

In [18]:
cbxgb.efficiency_

0.05533317299476564

In [ ]:
grid_searcher = RandomizedSearchCV(CutBasedXGBClassifier(nthread=1, mvaFeatures=classic_MVA_features,
                                                         n_estimators=300),
                                   {
        'P_cut': np.linspace(2, 5, 10),
        'PT_cut': np.linspace(0, 2, 10),
        'phiDistance_cut': np.linspace(0, 0.5, 10),
        'MuonPIDIsMuon_cut': [0, 1],
        'TRGHP_cut': np.linspace(0, 0.6, 10),
        'IPPUs_cut': np.linspace(1, 4, 10),
        'n_estimators': [200, 300, 400],
    }, n_iter=300, error_score=0, verbose=1, cv=skf, n_jobs=12)

In [ ]:
grid_searcher.fit(df, df.target)

Fitting 2 folds for each of 300 candidates, totalling 600 fits
Yielding split
Yielding split
Yielding split
Yielding split
Yielding split
Yielding split


| chunks | nfits | xgb jobs | grid search jobs | time/min |
|--------|-------|----------|------------------|------|
| 30     | 32    | 1        | 1                | 2.3  |
|        |       | 1        | 32               | 2.3  |
|        |       | 32       | 1                | 3.4  |
| all    | 32    | 1        | 32               | ?    |
|        | 2     | 1        | (32)             | 2.8  |
|        | 6     | 1        | (32)             | 6.1  |

In [ ]:
grid_searcher.best_score_ * 100

In [ ]:
grid_searcher.best_params_

In [ ]:
best_index = np.nanargmax([sc[1] for sc in grid_searcher.grid_scores_])
best_score = grid_searcher.grid_scores_[best_index]
print('tagging power {}% with params\n{}'.format(100 * ufloat(np.mean(best_score[2]), np.std(best_score[2])),
                                                 best_score[0]))
best_params = best_score[0]